In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
from data import gen_data, generate_random_edge_index, CustomData
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence,\
    pad_sequence, \
    pad_packed_sequence
from model import AE_rnn, AE_gnnrnn
import torch.optim as optim

In [3]:
data = gen_data(n_examples=1000,
                start_low=0, start_high=10,
                delta_low=1, delta_high=5,
                len_low=5, len_high=30)

In [10]:
bidir          = True
dir            = 2 if bidir else 1 
batch_size     = 64
num_layers_enc = 3
hidden_dim_enc = 5
num_layers_dec = 2
hidden_dim_dec = 4
emb_dim        = 7
N_max          = 256


ds = CustomData(data) #dataset


ae = AE_rnn(batch_size     = batch_size,
            bidir          = True,
            num_layers_enc = num_layers_enc,
            hidden_dim_enc = hidden_dim_enc,
            num_layers_dec = num_layers_dec,
            hidden_dim_dec = num_layers_dec,
            emb_dim        = emb_dim,
            N_max          = N_max)

In [ ]:
#AE rnn

if torch.cuda.is_available():
    device = torch.device("cuda")  # Or "cuda:0" for the first GPU
else:
    device = torch.device("cpu")

#device = torch.device("cpu")
ae.to(device)


data_gpu = []
for idx in range(0, len(ds), batch_size):
    seq = ds[idx:idx+batch_size]
    seq = [torch.from_numpy(s).to(device) for s in seq]
    padded_seq = pad_sequence(seq, batch_first=True)
    lengths = [len(s) for s in seq]
    #lengths = torch.tensor([len(s) for s in seq]).to(device)
    data_gpu.append((padded_seq, lengths))


criterion = nn.CrossEntropyLoss()
#data_emb_device = [(edge_index.to(device), [bb.to(device) for bb in node_embs ]) for (edge_index, node_embs) in data_emb] 
optimizer = optim.Adam(ae.parameters(), lr=0.01)
epoch_num = 100
for epoch in range(epoch_num):
    total_loss = 0
    total_items = 0
    j = -1
    for (padded_seq, lengths) in data_gpu:
        out = ae(padded_seq, lengths)
        loss = criterion(out.flatten(0).reshape(-1,N_max), padded_seq.flatten())
        total_loss  += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {total_loss:.5f}', end='\r')
    

In [6]:
bidir          = True
dir            = 2 if bidir else 1 
batch_size     = 256
num_layers_enc = 6
hidden_dim_enc = 16
num_layers_dec = 6
hidden_dim_dec = 16
emb_dim        = 32
N_max          = 256


ds = CustomData(data) #dataset


ae = AE_gnnrnn(batch_size     = batch_size,
            bidir          = True,
            num_layers_enc = num_layers_enc,
            hidden_dim_enc = hidden_dim_enc,
            num_layers_dec = num_layers_dec,
            hidden_dim_dec = num_layers_dec,
            emb_dim        = emb_dim,
            N_max          = N_max)

In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Or "cuda:0" for the first GPU
else:
    device = torch.device("cpu")

#device = torch.device("cpu")
ae.to(device)


data_gpu = []
for idx in range(0, len(ds), batch_size):
    seq = ds[idx:idx+batch_size]
    seq = [torch.from_numpy(s).to(device) for s in seq]
    padded_seq = pad_sequence(seq, batch_first=True)
    lengths = [len(s) for s in seq]
    edge_index = generate_random_edge_index(len(lengths)).to(device)
    #lengths = torch.tensor([len(s) for s in seq]).to(device)
    data_gpu.append((padded_seq, lengths, edge_index))


criterion = nn.CrossEntropyLoss()
#data_emb_device = [(edge_index.to(device), [bb.to(device) for bb in node_embs ]) for (edge_index, node_embs) in data_emb] 
optimizer = optim.Adam(ae.parameters(), lr=0.01)
epoch_num = 100
for epoch in range(epoch_num):
    total_loss = 0
    total_items = 0
    j = -1
    for (padded_seq, lengths, edge_index) in data_gpu:
        print(padded_seq.shape)
        out = ae(padded_seq, lengths, edge_index)
        loss = criterion(out.flatten(0).reshape(-1,N_max), padded_seq.flatten())
        total_loss  += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {total_loss:.5f}', end='\r')

torch.Size([256, 29])
0
1
2 torch.Size([12, 256, 16])
256
torch.Size([256, 29])
0
1
2 torch.Size([12, 256, 16])
256
torch.Size([256, 29])
0
1
2 torch.Size([12, 256, 16])
256
torch.Size([232, 29])
0
1
2 torch.Size([12, 232, 16])
232
torch.Size([256, 29])h 1/100, Average Loss: 22.27817
0
1
2 torch.Size([12, 256, 16])
256
torch.Size([256, 29])
0
1
2 torch.Size([12, 256, 16])
256
torch.Size([256, 29])
0
1
2 torch.Size([12, 256, 16])
256
torch.Size([232, 29])
0
1
2 torch.Size([12, 232, 16])
232
torch.Size([256, 29])h 2/100, Average Loss: 20.92874
0
1
2 torch.Size([12, 256, 16])
256
torch.Size([256, 29])
0
1
2 torch.Size([12, 256, 16])
256
torch.Size([256, 29])
0
1
2 torch.Size([12, 256, 16])
256
torch.Size([232, 29])
0
1
2 torch.Size([12, 232, 16])
232
torch.Size([256, 29])h 3/100, Average Loss: 19.28394
0
1
2 torch.Size([12, 256, 16])
256
torch.Size([256, 29])
0
1
2 torch.Size([12, 256, 16])
256
torch.Size([256, 29])
0
1
2 torch.Size([12, 256, 16])
256
torch.Size([232, 29])
0
1
2 torch.Siz